In [1]:
# Import packages from other direction. Itis necessary if the project is structured as:
# my_project
# ├── notebooks
# │   └── Generate Radio Data for tr.ipynb
# ├── local_python_package
# │   ├── __init__.py
# │   ├── models.py
# ├── README.md
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import tensorflow as tf
from radioml.dataset import RadioData

## Parameters

In [3]:
DATA_LEN = 200
PREAMBLE_LEN = 40
CHANNEL_LEN = 1

SNR_TRAIN = 20.0
OMEGA_TRAIN = 1/50

## Network Funcs

In [4]:
from tensorflow.keras.layers import Input, Dense, GRU, LSTM, RepeatVector
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras.layers import Lambda

tf.keras.backend.clear_session()

def cfo_network(preamble, preamble_conv, scope="CFOCorrectionNet"):
    """Build CFO Correction Network
    Arguments:
        preamble :     tf.Tensor float32 -  [batch, preamble_length, 2]
        preamble_conv: tf.Tensor float32 -  [batch, preamble_length, 2]
        
    Return:
        cfo_est: tf.Tensor float32 - [batch_size, 1]
    """
    with tf.name_scope(scope):
        inputs = tf.keras.layers.concatenate([preamble, preamble_conv], axis=-1,
                                            name='Preamble_PreambleConv')
        inputs = tf.keras.layers.Flatten()(inputs)
        x = Dense(100, 'tanh',  name=scope+"_dense_1")(inputs)
        x = Dense(100, 'tanh',  name=scope+"_dense_2")(x)
        x = Dense(100, 'tanh',  name=scope+"_dense_3")(x)
        cfo_est = Dense(1, 'linear', name='CFOEstimate')(x)
        
    return cfo_est


def equalization_network(cfo_corrected_packets, cfo_corrected_preamble, preamble):
    """Given a [preamble, preambe_conv] and """
    
    with tf.name_scope('ChannelEstimateNet'):
        inputs = tf.keras.layers.concatenate([preamble, cfo_corrected_preamble], axis=-1,
                                            name='Preamble_RotatedPreamble')
        inputs = tf.keras.layers.Flatten()(inputs)
        x = Dense(300, 'relu')(inputs)
        x = Dense(300, 'relu')(x)
        x = Dense(2, 'sigmoid', name='ChannelEsitate')(x)
        
    chan_est = RepeatVector(200)(x)
    inputs = tf.keras.layers.concatenate([cfo_corrected_packets, chan_est], axis=-1,
                                        name="CFOCorrected_ChannelEstimate")
    with tf.name_scope('EqualizationNet'):
        x = Bidirectional(LSTM(45, return_sequences=True))(inputs)
        x = Bidirectional(LSTM(45, return_sequences=True))(x)
        x = TimeDistributed(Dense(100, activation='relu'))(x)
        x = TimeDistributed(Dense(2, activation='linear'))(x)

    return x


def demod_and_ecc_network(equalized_packets):
    with tf.name_scope('DemodAndDecodeNet'):
        x = Bidirectional(LSTM(400, 'selu', return_sequences=True))(equalized_packets)
        x = Bidirectional(LSTM(400, 'selu', return_sequences=True))(x)
        
    data_estimates = TimeDistributed(Dense(1, activation='sigmoid'), name='DataEstimate')(x)
    
    return data_estimates

def cfo_correction_func(kwargs):
    """Rotate packets given an omega estimate 
    
    Arguments:
        omega_estimate: tf.Tensor float32 - [batch, 1]
        packets:        tf.Tensor float32 - [batch, (preamble_len + data_len), 2] 
        
    Return:
        rotated_packets: tf.Tensor float32 - [batch, (preamble_len + data_len), 2] 
    """ 
    # Because of Lambda Layer, we need to pass arguments as Kwargs
    omega_estimate, packets = kwargs[0], kwargs[1]

    # Build rotation matrix
    with tf.name_scope('rotation_matrix'):
        packet_len      = tf.cast(tf.shape(packets)[1], tf.float32) # preamble_len + data_len
        rotation_matrix = tf.exp(tf.complex(0.0, - 1.0 * omega_estimate * tf.range(packet_len)))

    # CFO Correction 
    with tf.name_scope('cfo_correction'):
        rotated_packets = tf.complex(packets[..., 0], packets[...,1]) * rotation_matrix

    # Encode complex packets into 2D array
    with tf.name_scope('corrected_preamble'):
        corrected_preambe = tf.stack([tf.real(rotated_packets)[..., :PREAMBLE_LEN], 
                                      tf.imag(rotated_packets)[..., :PREAMBLE_LEN]], 
                                      axis=-1)
    with tf.name_scope('rotated_packets'):
        rotated_packets = tf.stack([tf.real(rotated_packets)[..., PREAMBLE_LEN:], 
                                      tf.imag(rotated_packets)[..., PREAMBLE_LEN:]], 
                                      axis=-1)
        
    return corrected_preambe, rotated_packets

##  Build a  Gigantic Model!

In [5]:
preamble          = Input(shape=(40, 2), name='preamble')
preamble_conv     = Input(shape=(40, 2), name='preamble_conv')
corrupted_packets = Input(shape=(240, 2), name='corrupted_packets')  # [preamble_conv, data_conv]

 
cfo_est = cfo_network(preamble, preamble_conv)
cfo_corrected_preamble, cfo_corrected_packets = \
    Lambda(cfo_correction_func,name='CFO_Correction')(
            [cfo_est, corrupted_packets])

equalized_packets = equalization_network(cfo_corrected_packets, cfo_corrected_preamble, preamble)
data_estimates    = demod_and_ecc_network(equalized_packets)

model = tf.keras.Model([corrupted_packets, preamble, preamble_conv], data_estimates)

print("Number of training parameters: %d" % model.count_params())

Number of training parameters: 5385466


In [6]:

from IPython.display import SVG, display
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
%matplotlib inline
SVG(model_to_dot(model, show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))

NameError: name 'SVG' is not defined

In [ ]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:800px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:820px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
# show_graph(tf.get_default_graph().as_graph_def())